In [20]:
b = 10
num_blocks = 100
n = num_blocks*(b-1) + 1

A = spzeros(n, n)

for i = 1:num_blocks
    top_left = (i-1)*(b-1) + 1
    A[top_left:top_left+b-1, top_left:top_left+b-1] .= randn(b, b)
end

A = (A + A')/2;

B = speye(n, n);

In [21]:
@time eigs(A, nev=1, which=:SR)

  0.009239 seconds (800 allocations: 212.766 KiB)


([-5.59754], [-8.31145e-19; 2.61561e-17; … ; 2.73608e-17; -2.81534e-18], 1, 9, 100, [-0.102409, 0.14521, 0.195165, 0.125402, 0.031822, -0.00651783, 0.279918, -0.0213263, -0.197338, -0.434905  …  0.000944501, -0.0327124, -0.0516399, 0.0537226, -0.0286441, 0.111385, 0.0989107, -0.0696658, 0.0281797, 0.0511693])

In [22]:
# grad(chol_L, B) = -1 - trace(chol_L \ B)
@time begin

function grad(chol_L::SparseArrays.CHOLMOD.Factor{Float64}, B::SparseMatrixCSC{Float64, Int64}, nu::Float64)
    tr = 0
    e_i = zeros(size(B, 2))
    e_i[1] = 1
    for i = 1:size(B, 2)
        if i > 1
            e_i[i-1] = 0
            e_i[i] = 1
        end
        tr += (chol_L \ e_i)[i]
    end
    return -1 + nu*tr
end
    
eval_f(chol_L::SparseArrays.CHOLMOD.Factor, t::Float64, nu::Float64) = -t - nu*logdet(chol_L)

curr_t = -100.
step_size = 1
nu = 1.

L = A - curr_t*B
init_chol_L = cholfact(L)
chol_L = copy(init_chol_L)

converged = false
    
prev_val = Inf
    
for i = 1:100
    curr_grad = grad(chol_L, B, nu)
    is_pos_def = false
    while !is_pos_def
        tent_t = curr_t - step_size*curr_grad
        L .= A - tent_t*B

        try
            cholfact!(chol_L, L)
            
            curr_eval = eval_f(chol_L, tent_t, nu)
            
            if curr_eval > prev_val
                step_size *= .5
                continue
            end
            
            prev_val = curr_eval
            is_pos_def = true
                
            if abs(curr_grad)^2 < 1e-4
                println("Curr t : $tent_t")
                if nu < 1e-4
                    converged = true
                    break
                end
                
                nu *= .5
                prev_val = Inf
                break
            end
                
            curr_t = tent_t
            step_size *= 1.2
            break

        catch y
            if isa(y, Base.LinAlg.PosDefException)
                chol_L = copy(init_chol_L)
                step_size *= .5
            else
                error("wtf?")
            end
        end
    end
    
    if converged
        break
    end
end
    
end

Curr t : -899.1288056022731
Curr t : -451.2202204483291
Curr t : -225.49410257896068
Curr t : -112.96890241374686
Curr t : -56.142890024639584
Curr t : -28.392589892218503
Curr t : -14.545908912667237
Curr t : -7.926743561411388
Curr t : -5.645263886603386
Curr t : -5.602060265564332
Curr t : -5.59892936755401
  1.775434 seconds (1.28 M allocations: 4.349 GiB, 8.84% gc time)


In [18]:
eigs(A,nev=1,which=:SR)

([-6.17813], [-8.86227e-19; 1.70165e-18; … ; -7.09836e-17; 1.31684e-18], 1, 9, 100, [-0.00832226, -0.00582737, -0.0372743, 0.0239053, 0.0278231, 0.00543517, 0.0221169, -0.00404483, -0.0160135, -0.00235164  …  0.047227, -0.0434156, -0.00583877, 0.0929452, -0.054085, 0.0507873, -0.0536809, -0.0842467, -0.109428, 0.07352])

In [14]:
nnz(chol_L\B)

2940308

In [74]:
func(x, t, nu) = -t - nu*log(x - t)
grad(x, t, nu) = -1 + nu/(x - t)

curr_t = -100
step_size = 1

x_set = 1
nu = 1
epsilon = 1e-6
prev_val = Inf

for i=1:100
    curr_grad = grad(x_set, curr_t, nu)
    tent_t = curr_t - step_size*curr_grad
    
    if tent_t > x_set
        step_size *= .5
        continue
    end
    
    tent_val = func(x_set, tent_t, nu)
    
    if tent_val > prev_val
        step_size *= .5
        continue
    end
    
    prev_val = tent_val
    
    if abs(curr_grad)^2 < epsilon
        println("Finished optimizing iteration at nu : $nu with t $tent_val")
        if nu < 1e-4
            break
        end
        prev_val = Inf
        nu *= .1
    end
    
    curr_t = tent_t
    
    step_size *= 1.2
end

Finished optimizing iteration at nu : 1 with t 1.0484698606752908e-9
Finished optimizing iteration at nu : 0.1 with t -0.669741490289786
Finished optimizing iteration at nu : 0.010000000000000002 with t -0.9439482969657267
Finished optimizing iteration at nu : 0.0010000000000000002 with t -0.9920922447141156
Finished optimizing iteration at nu : 0.00010000000000000003 with t -0.9989789659628019
Finished optimizing iteration at nu : 1.0000000000000004e-5 with t -0.9998748707422735


In [75]:
Pkg.update("Convex")

INFO: Updating METADATA...
INFO: Updating cache of DataDeps...
INFO: Updating cache of VersionParsing...
INFO: Updating cache of ZMQ...
INFO: Updating cache of HTTP...
INFO: Updating cache of Conda...
INFO: Computing changes...
INFO: Upgrading Compat: v0.67.0 => v0.68.0
- Compat
Restart Julia to use the updated versions.
